Densnet主要由dense block构成，下面来实现一个dense block 

In [3]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

定义一个卷积块，顺序为bn -> relu -> conv

In [4]:
def conv_block(in_channel, out_channel):
    layer = nn.Sequential(
        nn.BatchNorm2d(in_channel),
        nn.ReLU(True),
        nn.Conv2d(in_channel,out_channel,3,padding=1,bias=False)
    )
    return layer

dense block将每次的卷积输出称为 growth_rate,因为如果输入是in_channel,有n层，那么输出就是 in_channel + n * growth_rate

In [8]:
class dense_block(nn.Module):
    def __init__(self, in_channel, growth_rate, num_layers):
        super(dense_block, self).__init__()
        block = []
        channel = in_channel
        for i in range(num_layers):
            block.append(conv_block(channel, growth_rate))
            channel += growth_rate
            
        self.net = nn.Sequential(*block)
        
    def forward(self, x):
        for layer in self.net:
            out = layer(x)
            x = torch.cat((out, x), dim=1)
        return x

In [9]:
test_net = dense_block(3, 12, 3)
test_x = Variable(torch.zeros(1, 3, 96, 96))
print('input shape:{} x {} x {}'.format(test_x.shape[1],test_x.shape[2],test_x.shape[3]))
test_y=test_net(test_x)
print('output shape:{} x {} x {}'.format(test_y.shape[1],test_y.shape[2],test_y.shape[3]))

input shape:3 x 96 x 96
output shape:39 x 96 x 96
